ANALISIS EXPLORATORIO DE DATOS --- DATASET CS:GO

Fase 1 -- Comprender el negocio

En el siguiente EDA haremos un analisis extensivo de conjunto de datos de jugadores de cs:go, analizando estadisticas de forma que podamos recopilar informacion valiosa acerca de como ciertos parametros pueden afectar al rendimiento del jugador, asi como obtener informacion que pueda ser de utilidad para tener un panorama claro de como ciertas estadisticas siguen ciertos patrones para propositos de estudio


Fase 2 --- Comprender los datos
En este estudio nos haremos dos preguntas
¿Podemos determinar las kills que tendra un jugador en una partida por sus diferentes metricas?
¿Podemos determinar si el impacto del jugador es Positivo o Negativo por medio de un conjunto de metricas correlacionadas en la partida?

In [ ]:
#importar librerias necesarias
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from google.colab import files

In [ ]:
#cargar datos
uploaded = files.upload()
for fn in uploaded.keys():
    name = fn

Saving Anexo ET_demo_round_traces_2022.csv to Anexo ET_demo_round_traces_2022.csv


In [ ]:
df = pd.read_csv(name, sep=";")


-- Exploracion inicial de datos

In [ ]:
#cantidad filas, columnas
df.shape

In [ ]:
#nombre de las columnas
df.columns

In [ ]:
#Muestra las primeras 5 filas para comprobar si los datos cargaron correctamente
display(df.head())

In [ ]:
#Muestra qué tipo de datos hay, si es texto, número, booleano y cuántos datos faltan.
df.info()

In [ ]:
#5 primeras filas de las columnas
df.head()

In [ ]:
#resumen general dataframe
df.info()

In [ ]:
#visualizacion de datos nulos
df.isna().sum()
#solo dos filas presentan datos nulos, junto a esto tambien esta una columna con error de carga (unnamed:0)

In [ ]:
#analisis estadistico basico
df.describe().T

Por parte del analisis anterior se pueden observar datos interesantes para entender el comportamiento de los jugadores

Por parte de la compra de armas
La gran mayoria usa armas de asalto como arma primaria, asi como que muy pocos usan armas pesadas o smg
Muy pocos compran granadas de daño, la gran mayoria solo compra utilidad

Por parte del los jugadores por partida
El promedio hace alrededor de 9 kills por partida, siendo 4 de estas con disparo a la cabeza


Limpieza

In [ ]:
#verificacion de datos a los extremos
df["headshot_percentage"] = df["RoundHeadshots"] / df["RoundKills"]

In [ ]:
df['TimeAlive'] = pd.to_numeric(df['TimeAlive'], errors='coerce')
df['TravelledDistance'] = pd.to_numeric(df['TravelledDistance'], errors='coerce')
df['FirstKillTime'] = pd.to_numeric(df['FirstKillTime'], errors='coerce')

In [ ]:
df['headshot_percentage'] = df['RoundHeadshots'] / df['RoundKills']

In [ ]:
df['ImpactPlayerScore'] = df['RoundKills'] + 0.5 * df['RoundAssists'] + 1.5 * df['RoundHeadshots']

In [ ]:
# Se hacen las respectivas correlaciones entre variables
df_encoded = pd.get_dummies(df, columns=["Map", "Team"], drop_first=True)
df_numeric = df_encoded.select_dtypes(include=['float64', 'int64'])
corr_matrix = df_numeric.corr()
corr_matrix["MatchKills"].sort_values(ascending=False)


Las anteriores correlaciones nos indican que los headshots son la estadistica que mas hace aumentar la cantidad de kills por partida, junto a esto le sigue el tener mayor cantidad de rondas ya que habria mas tiempo para realizar mas kills asi como que las kills por flanqueo tambien tienen una alta correlacion con la cantidad de kills por partida, en resumen realizar headshots y flanquear aumentan la cantidad de kills significativamente

In [ ]:
#Dibuja un histograma para ver cuántos kills por ronda suelen hacer los jugadores.
sns.histplot(df['RoundKills'], kde=True)


In [ ]:
#verificacion de posibles correlaciones en forma visual
sb.pairplot(df[["MatchKills", "RoundKills", "RoundHeadshots", "RoundAssists"]])
plt.suptitle("Pairplot de MatchKills, RoundKills, RoundHeadshots y RoundAssists", y=1.02)
plt.show()

In [ ]:
#Dibuja un boxplot para comparar kills entre los que sobreviven y los que mueren.
sns.boxplot(x='Survived', y='RoundKills', data=df)

In [ ]:
#Este gráfico sirve para identificar fácilmente qué variables están relacionadas entre sí y qué tan fuerte es esa relación.
plt.figure(figsize=(18,14))
sb.heatmap(corr_matrix, cmap="coolwarm", vmin=-1, vmax=1, annot=True, fmt=".2f")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.title("Mapa de Calor de Correlaciones", fontsize=18)
plt.show()

Fase 3 --- preparacion de los datos

In [ ]:
df_encoded = pd.get_dummies(df, columns=["Map", "Team"], drop_first=True)

for col in ['TimeAlive', 'TravelledDistance', 'ImpactPlayerScore']:
    if col in df_encoded.columns:
        df_encoded[col] = pd.to_numeric(df_encoded[col], errors='coerce')

df_numeric = df_encoded.select_dtypes(include=['float64', 'int64']).copy()

vars_to_scale = [col for col in ['TimeAlive', 'TravelledDistance', 'RoundKills', 'MatchKills', 'ImpactPlayerScore'] if col in df_numeric.columns]

scaler = StandardScaler()
df_numeric[vars_to_scale] = scaler.fit_transform(df_numeric[vars_to_scale])

if 'RoundKills' in df_numeric.columns and 'TimeAlive' in df_numeric.columns:
    df_numeric['Kills_per_minute'] = df_numeric['RoundKills'] / (df_numeric['TimeAlive'] / 60)
    df_numeric['Kills_per_minute'] = df_numeric['Kills_per_minute'].fillna(0)


In [ ]:
df_numeric.describe()
#un fragmento de codigo para ver la nueva variable